In [1]:
import numpy as np
import trimesh
import tqdm 

In [2]:
def uniform_sampling_from_mesh(vertices, faces, sample_num):
    # -------- TODO -----------
    # 1. compute area of each triangles
    # 2. compute probability of each triangles from areas
    # 3. sample N faces according to the probability
    # 4. for each face, sample 1 point
    # Note that FOR-LOOP is not allowed!
    # -------- TODO -----------
	
	# Step 1: compute area of each triangles
	area = np.cross(vertices[faces[:, 1]] - vertices[faces[:, 0]], vertices[faces[:, 2]] - vertices[faces[:, 1]]) / 2.0
	area = np.linalg.norm(area, axis=1)
	area = np.fabs(area)
	
	# Step 2: compute probability of each triangles from areas
	prob = area / np.sum(area)
	
	# Step 3: sample N faces according to the probability
	sampled_faces = np.random.choice(len(faces), size=sample_num, replace=True, p=prob)

	# Step 4: for each face, sample 1 point
	w1 = np.random.rand(sample_num, 1)
	w2 = np.random.rand(sample_num, 1)
	w = np.concatenate([w1, w2], axis=1)
	mask = np.where(np.sum(w, axis=1) > 1)
	w[mask] = 1 - w[mask]
	w3 = 1 - np.sum(w, axis=1, keepdims=True)
	w = np.concatenate([w, w3], axis=1)

	uniform_pc = np.sum(vertices[faces[sampled_faces]]*np.expand_dims(w, axis=2), axis=1)

	return area, prob, uniform_pc
        

In [3]:
def farthest_point_sampling(pc, sample_num):
    # -------- TODO -----------
    # FOR LOOP is allowed here.
    # -------- TODO -----------

	results = []
	results.append(np.random.randint(0, len(pc)))
	for i in range(sample_num-1):
		max_idx = -1

		mask = np.ones(len(pc), dtype=bool)
		mask[results] = False

		distance = np.linalg.norm(np.expand_dims(pc,1) - np.expand_dims(pc[results],0), axis=-1)
		min_dist = np.min(distance, axis=1)
		max_idx = np.argmax(min_dist)

		results.append(max_idx)

	return pc[results]

In [4]:
# task 1: uniform sampling 

obj_path = 'spot.obj'
mesh = trimesh.load(obj_path)
print('faces shape: ', mesh.faces.shape)
sample_num = 512
area, prob, uniform_pc = uniform_sampling_from_mesh(mesh.vertices, mesh.faces, sample_num)

# Visualization. For you to check your code
np.savetxt('uniform_sampling_vis.txt', uniform_pc)

print('area shape: ',area.shape)
print('prob shape: ',prob.shape)
print('pc shape: ',uniform_pc.shape)
# the result should satisfy: 
#       area.shape = (13712, ) 
#       prob.shape = (13712, ) 
#       uniform_pc.shape = (512, 3) 

# For submission
save_dict = {'area': area, 'prob': prob, 'pc': uniform_pc}
np.save('../results/uniform_sampling_results', save_dict)

faces shape:  (13712, 3)
area shape:  (13712,)
prob shape:  (13712,)
pc shape:  (512, 3)


In [5]:
# task 2: FPS

init_sample_num = 2000
final_sample_num = 512
_,_, tmp_pc = uniform_sampling_from_mesh(mesh.vertices, mesh.faces, init_sample_num)
fps_pc = farthest_point_sampling(tmp_pc, final_sample_num)

# Visualization. For you to check your code
np.savetxt('fps_vis.txt', fps_pc)

# For submission
np.save('../results/fps_results', fps_pc)

In [6]:
# task 3: metrics

from scipy.optimize import linear_sum_assignment
from scipy.spatial.distance import cdist

# from earthmover.earthmover import earthmover_distance   # EMD may be very slow (1~2mins)
# -----------TODO---------------
# compute chamfer distance and EMD for two point clouds sampled by uniform sampling and FPS.
# sample and compute CD and EMD again. repeat for five times.
# save the mean and var.
# -----------TODO---------------

CD_mean = 0
CD_var = 0
EMD_mean = 0
EMD_var = 0
CD = []
EMD = []

init_sample_num = 2000
final_sample_num = 512

def chamfer_distance(pc1, pc2):
	distance_matrix = np.linalg.norm(np.expand_dims(pc1, 1) - np.expand_dims(pc2, 0), axis=-1)
	distance_1 = np.mean(np.min(distance_matrix, axis=1))
	distance_2 = np.mean(np.min(distance_matrix, axis=0))	
	return distance_1 + distance_2

def earthmover_distance(pc1, pc2):
	cost_matrix = cdist(pc1, pc2, metric='euclidean')
	row_ind, col_ind = linear_sum_assignment(cost_matrix)
	return cost_matrix[row_ind, col_ind].mean()

for i in range(5):
	_,_,uniform_sampled_pc = uniform_sampling_from_mesh(mesh.vertices, mesh.faces, sample_num=final_sample_num)
    
	_,_, tmp_pc = uniform_sampling_from_mesh(mesh.vertices, mesh.faces, init_sample_num)
	fps_pc = farthest_point_sampling(tmp_pc, final_sample_num)
      
	CD.append(chamfer_distance(uniform_sampled_pc, fps_pc))
	EMD.append(earthmover_distance(uniform_sampled_pc, fps_pc))

CD_mean = np.mean(CD)
CD_var = np.var(CD)
EMD_mean = np.mean(EMD)
EMD_var = np.var(EMD)

# For submission
np.save('../results/metrics', {'CD_mean':CD_mean, 'CD_var':CD_var, 'EMD_mean':EMD_mean, 'EMD_var':EMD_var})